In [1]:
import pandas as pd
import numpy as np

In [2]:
import configparser

config = configparser.ConfigParser()
config.read("./config.ini")

['./config.ini']

In [3]:
city = "Delhi"

In [4]:
stations_df = pd.read_csv('./stations.csv')
stations_df.head()

,StationId,StationName,City,State,Status
0,AP001,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
1,AP002,"Anand Kala Kshetram, Rajamahendravaram - APPCB",Rajamahendravaram,Andhra Pradesh,NaN
2,AP003,"Tirumala, Tirupati - APPCB",Tirupati,Andhra Pradesh,NaN
3,AP004,"PWD Grounds, Vijayawada - APPCB",Vijayawada,Andhra Pradesh,NaN
4,AP005,"GVM Corporation, Visakhapatnam - APPCB",Visakhapatnam,Andhra Pradesh,Active


In [5]:
city_stations_df = stations_df[(stations_df['City'] == city) & (stations_df['Status'] == 'Active')][['StationId', 'StationName', 'City']].reset_index(drop=True)
city_stations_df.head()

,StationId,StationName,City
0,DL001,"Alipur, Delhi - DPCC",Delhi
1,DL002,"Anand Vihar, Delhi - DPCC",Delhi
2,DL003,"Ashok Vihar, Delhi - DPCC",Delhi
3,DL004,"Aya Nagar, Delhi - IMD",Delhi
4,DL005,"Bawana, Delhi - DPCC",Delhi


In [6]:
city_stations_df.tail()

,StationId,StationName,City
32,DL034,"Sirifort, Delhi - CPCB",Delhi
33,DL035,"Sonia Vihar, Delhi - DPCC",Delhi
34,DL036,"Sri Aurobindo Marg, Delhi - DPCC",Delhi
35,DL037,"Vivek Vihar, Delhi - DPCC",Delhi
36,DL038,"Wazirpur, Delhi - DPCC",Delhi


In [7]:
import googlemaps

gmaps = googlemaps.Client(key=config['GMaps']['api_key'])

def get_lat_lon(place):
    geocode_result = gmaps.geocode(place + f', {city}, India')
    result = geocode_result[0]['geometry']['location']
    return result['lat'], result['lng']

In [8]:
geocode_rows = []

for station_name in city_stations_df['StationName'].values:
    try:
        lat, lon = get_lat_lon(station_name)
        row = (station_name, lat, lon)
        geocode_rows.append(row)
    except:
        pass

In [9]:
geocode_df = pd.DataFrame(geocode_rows, columns=['StationName', 'Latitude', 'Longitude'])

In [10]:
city_stations_df = pd.concat([city_stations_df.set_index('StationName'), geocode_df.set_index('StationName')], axis=1).reset_index()
city_stations_df.columns = ['StationName', 'StationId', 'City', 'Latitude', 'Longitude']

In [11]:
city_stations_df.head()

,StationName,StationId,City,Latitude,Longitude
0,"Alipur, Delhi - DPCC",DL001,Delhi,28.797226,77.133136
1,"Anand Vihar, Delhi - DPCC",DL002,Delhi,28.647634,77.315862
2,"Ashok Vihar, Delhi - DPCC",DL003,Delhi,28.668722,77.229979
3,"Aya Nagar, Delhi - IMD",DL004,Delhi,28.472044,77.132942
4,"Bawana, Delhi - DPCC",DL005,Delhi,28.822938,77.102007


In [18]:
city_stations_df.dropna().reset_index(drop=True).to_pickle(f'./{city.lower()}_stations.pkl', protocol=4)